In [63]:
# importing the requests library
import requests
from cvxopt import matrix, solvers
import json
import datetime
from dateutil.relativedelta import *
from forex_python.converter import CurrencyRates
import numpy as np
import re
solvers.options['show_progress'] = False

In [188]:
import requests
import json
class RequestManadger:
    def __init__(self, url,  user = '', password = '',auth = True):
        self.url = url
        self.session = requests.Session()
        self.ids = []
        if (auth):
            if ((user != '') and (password != '')):
                self.user = user
                self.session.auth = (user, password)
                self.session.headers.update({'x-test': 'true'})
                r = self.session.get(self.url, verify = False)
                self.load_datas()
                
            else :
                print("you should precise explicitly auth = false if "
                      "you dont want add a user name and a password")
        else:
            return self.session.get(self.url, verify = False)
    def load_datas(self):
        datas = {'ids':'my_ids.txt',"ass":"my_assets.txt",
                 "corr":"my_corr.txt",'ass65':"my_assets65.txt",
                 "var":"my_vars.txt","shp65":"my_sharpes65.txt",
                 "cov":"my_cov.txt","shp":"my_sharpes.txt",
                 "prx_cur":"my_prx_curr.txt"}
        
        self.ids = self.read_file(datas['ids']).astype(int)
        self.assets = self.read_file(datas['ass'])
        self.sharpes = self.read_file(datas['shp'])
        self.assets65 = self.read_file(datas['ass65'])
        self.sharpes65 = self.read_file(datas['shp65'])
        self.prx_cur = self.read_file(datas['prx_cur'], prx_cur = True).T
        self.vars = self.read_file(datas['var'])
        self.corr =  self.read_file(datas['corr'])
        self.cov =  self.read_file(datas['cov'])
        self.cov2 = np.cov(self.assets65)
        self.corr2 = np.array([[self.cov2[i][j]/np.sqrt(np.var(self.assets65[i]) * np.var(self.assets65[j])) for j in range(self.cov2.shape[0])] for i in range(self.cov2.shape[1])])
        #data1 = f3.readlines() 
        #self.ret65 = np.array([np.array(data1[i].split()).astype(np.float64) for i in range(len(data1))])
        #data2 = f4.readlines()
        #self.cov = np.array([np.array(data2[i].split()).astype(np.float64) for i in range(len(data2))])
        
    def get_request(self,req):
        r = self.session.get(self.url + "/" + req, verify = False) 
        return json.loads(r.text)
    def get_assetsids(self):
        r = self.session.get(self.url + "/asset" , verify = False)
        dic = json.loads(r.text)
        N = len(dic)
        ass_ids = np.zeros(N)
        for i in range(N):
            ass_ids[i] = int(dic[i]["ASSET_DATABASE_ID"]["value"])
        self.ids = ass_ids.astype(int)
        self.write_file(self.ids, "my_ids.txt")
        return self.ids
    def get_asset_vecrs(self, ids, jump = 1, allinone = False):
        start = datetime.datetime(2012,1,1)
        end = datetime.datetime(2017,6,1)
        months = (end.year - start.year) * 12 + (end.month - start.month)
        asset_vecr = []
        asset_vecs =[]
        
        if not(allinone):
            end =  start + relativedelta(months=+jump)
        else:
            months = 1

        
        for i in range(months):
            obj2 = {'ratio':[21,20],'asset':[int(ids[i]) for i in range(len(ids))],'start_date':start.isoformat(),'end_date':end.isoformat(),'frequency':None}
            obj = json.dumps(obj2)
            r = self.session.post(self.url + "/ratio/invoke", data = obj, verify = False)
            dic = json.loads(r.text)
            asset_vecr.append([float((dic[str(ids[i])]['21']['value']).replace(",",'.')) for i in range(len(ids))])
            asset_vecs.append([float((dic[str(ids[i])]['20']['value']).replace(",",'.')) for i in range(len(ids))])
            
           # months = (end.year - start.year) * 12 + (end.month - start.month)
            start = start + relativedelta(months=+jump)
            end = end + relativedelta(months=+jump)
        asset_vecr = np.array(asset_vecr)
        asset_vecs = np.array(asset_vecs)
        if not(allinone):
            self.write_file(asset_vecr.T, 'my_assets65.txt')
            self.write_file(asset_vecs.T, 'my_sharpes65.txt')
        else:
            self.write_file(asset_vecr, 'my_assets.txt')
            self.write_file(asset_vecs, 'my_sharpes.txt')
            
        return asset_vecr, asset_vecs
    def write_file(self, data, name):
        f1 = open(name, "w")
        if (len(data.shape) == 2):
            for i in range(data.shape[0]):
                for j in range(data.shape[1] - 1):
                    f1.write(str(data[i][j]) + ' ')
                f1.write(str(data[i,data.shape[1] - 1]) + '\n')
        else:
            for i in range(data.shape[0] - 1):
                    f1.write(str(data[i]) + ' ')
            f1.write(str(data[data.shape[0] - 1]) + '\n')
        f1.close()
    def read_file(self, name, prx_cur = False):
        f1 = open(name, "r")
        data = f1.readlines()
        #print(data)
        f1.close()
        if (len(data) == 1):
            return np.array(data[0].split()).astype(np.float32)
        else:
            if(prx_cur):
                data= np.array([np.array(data[i].split()) for i in range(len(data))])
                return np.array([data[:,0].astype(np.float64), data[:,1]])
            else:
                return np.array([np.array(data[i].split()).astype(np.float64) for i in range(len(data))])
    
    
        
    def corr_matrix(self):
        ids = self.ids
        start = datetime.datetime(2012,1,1)
        end = datetime.datetime(2017,6,1)
        asset_corr = []
        for j in range(len(ids)):
            obj2 = {'ratio':[19],'asset':[int(ids[i]) for i in range(len(ids))],'benchmark':int(ids[j]),'start_date':start.isoformat(),'end_date':end.isoformat(),'frequency':None}
            obj = json.dumps(obj2)
            r = self.session.post(self.url + "/ratio/invoke", data = obj, verify = False)
            dic = json.loads(r.text)
            asset_corr.append([float((dic[str(ids[i])]['19']['value']).replace(",",'.')) for i in range(len(ids))])
            self.corr_mat = np.array(asset_corr)
            self.write_file(self.corr_mat, 'my_corr.txt')
        return self.corr_mat
    def get_variances(self):
        start = datetime.datetime(2012,1,1)
        end = datetime.datetime(2017,6,1)
        ids = self.ids
        var = []
        obj2 = {'ratio':[18],'asset':[int(ids[i]) for i in range(len(ids))],'start_date':start.isoformat(),'end_date':end.isoformat(),'frequency':None}
        obj = json.dumps(obj2)
        r = self.session.post(self.url + "/ratio/invoke", data = obj, verify = False)
        dic = json.loads(r.text)
        var = [float((dic[str(ids[i])]['18']['value']).replace(",",'.'))**2 for i in range(len(ids))]
        self.var = np.array(var)
        self.write_file(self.var, 'my_vars.txt')
        return self.var
    def get_ass_curr(self):
        r = self.get_request("asset")
        prices = []
        currencys = []
        for i in range(len(r)):
            a = r[i]['LAST_CLOSE_VALUE']['value']
            l = np.array(re.split(" |,", a))
            price = [l[i]  for i in range(len(l)) if (l[i].isnumeric())]
            currency  = r[i]['CURRENCY']['value']
            if(len(price) == 2):
                price = float(price[0] + '.' + price[1])
            else :
                price = float(price[0])
            prices.append(price)
            currencys.append(currency)
        self.prx_cur = np.array([np.array(prices), np.array(currencys)]).T
        self.write_file(self.prx_cur,'my_prx_curr.txt')
        return self.prx_cur



    
    def get_cov(self):
        ids = self.ids
        if (self.ids == []):
            self.ids = self.get_assetsids()
        if (self.corr_mat == []):
            self.corr_mat = self.corr_matrix(self, self.ids)
        self.cov = np.array([[self.corr_mat[i][j] / np.sqrt(self.var[i] * self.var[j]) for i in range(self.corr_mat.shape[0])]for j in range(self.corr_mat.shape[1]) ])
        self.write_file(self.cov, 'my_cov.txt')
        return self.cov

    def get_portfoliosh(self, ids, Ret_J = False, Sig_J = True):
        assert(len(ids) ==20), 'the lentgh of indices should be exactly 20'
        w = self.get_weights(ids, Ret_J = Ret_J, Sig_J = Sig_J) 
        Sigma = []
        RT = []
        if not(Ret_J):
            returns = self.assets65[ids]
            RT = np.mean(returns, axis=1).reshape(20,1)
        else :
            RT = self.assets[ids].reshape(20,1)
        if (Sig_J):
            Sigma = self.cov[ids][:,ids]
        else :
            Sigma = self.cov2[ids][:,ids]
        
        rp =  RT.T.dot(w)
        sigmap = np.sqrt(w.T.dot(Sigma.dot(w)))
        sharp = rp/sigmap
        return np.asscalar(sharp)
    def get_asset_quant(self, prx, cur, ammount):
        
        c = CurrencyRates()
       # print(cur)
        pr = float(prx)
        if (cur == 'EUR'):
            return int(ammount/ pr)
        else:
            val = c.convert(cur,'EUR',pr)
            return int(ammount/val)
    def get_assets_quant(self, ids, w, big_amount):
        assert(len(ids) == len(w)), 'weights and ids are not the same length'
        pc = self.prx_cur
        
        assets_quant = np.array([self.get_asset_quant(pc[i][0], pc[i][1] ,np.asscalar(big_amount * w[i])) for i in range(len(ids))])
        return assets_quant
    def get_weights(self, ids, Ret_J=False, Sig_J= True):
        assert(len(ids) == 20), 'the lentgh of indices should be exactly 20' 
        Sigma = []
        RT = []
        if not(Ret_J):
            returns = self.assets65[ids]
            RT = np.mean(returns, axis=1).reshape(20,1).astype(np.double)
        else :
            RT = self.assets[ids].reshape(20,1).astype(np.double)
        if (Sig_J):
            Sigma = self.cov[ids][:,ids]
        else :
            Sigma = self.cov2[ids][:,ids]
            
        N = 20
        P = matrix(Sigma)
        q = matrix(-2 * RT,tc='d')
        G = matrix(np.vstack((np.identity(N), -np.identity(N))),tc='d')

        h = matrix(np.hstack((0.1 * np.ones(N), np.ones(N) * -0.01)), tc='d')
        A = matrix(np.ones(N), (1,N)) 
        b = matrix([1.0])
        sol = solvers.qp(P, q, G, h, A, b)
        W = sol['x']
        return np.array(W)
            
        
            

In [189]:
URL = 'https://dolphin.jump-technology.com:3389/api/v1'
user = 'epita_user_5'
pwd = 'dolphin23235'
rm = RequestManadger(URL,user, pwd)



/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [158]:
f1 = open("my_sharpes.txt","r")
sharpes = np.array(f1.readline().split()).astype(np.float32)
f1.close()
idx = np.argsort(sharpes)[::-1]
ids = idx[:20]
sh1 = rm.get_portfoliosh(ids,Ret_J=True, Sig_J=True)
sh2 = rm.get_portfoliosh(ids, Ret_J=True, Sig_J=False)
sh3 = rm.get_portfoliosh(ids, Ret_J=False, Sig_J=True)
sh4 = rm.get_portfoliosh(ids, Ret_J=False, Sig_J=False)
print("Use Jump Return + Jump Sigma  :", sh1)
print("Use Jump Return + 65 Sigma  :", sh2)
print("Use 65 Return +  Jump Sigma :", sh3)
print("Use 65 Return + 65 Sigma :", sh4)
print("Sum of sharpes :", sh1 + sh2 + sh3 + sh4)



Use Jump Return + Jump Sigma  : 0.20161985138146446
Use Jump Return + 65 Sigma  : 80.65770457071
Use 65 Return +  Jump Sigma : 0.001624921225077248
Use 65 Return + 65 Sigma : 0.6535830034613018
Sum of sharpes : 81.51453234677784


In [168]:
w = rm.get_weights(ids)
w.shape

(20, 1)

In [160]:
type(rm.prx_cur[0,0])

numpy.str_

In [190]:
rm.get_assets_quant(ids,w , 1e7)

array([ 8068,  5159,  2380,  5583,   791,   696,   763,  1512,    45,
        5785,  1357,   915, 12793,  9254,  4902,  4101,  7543, 13315,
         555,  3244])

In [101]:
rm.get_asset_quant(200, 'USD',100000)

589

In [176]:
ret65, sh65 = rm.get_asset_vecrs(rm.ids)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [177]:
ret, sh = rm.get_asset_vecrs(rm.ids, allinone=True)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [185]:
var = rm.get_variances()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [187]:
corr = rm.corr_matrix()


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [197]:
rm.var = var
rm.corr_mat = corr
cov = rm.get_cov()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:146: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:148: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [104]:
data = np.array([prx, curs])
data.shape
write_file(data.T, 'price_currency.txt')

In [199]:
datas = {'ids':'my_ids.tx','ass65':"my_assets65.txt"}
datas

{'ass65': 'my_assets65.txt', 'ids': 'my_ids.tx'}

In [ ]:
def get_asset_quant(prx, cur ammount):
    c = CurrencyRates()
    if (cur == 'EUR'):
        return int(amount/ prx)
    else:
        return int(amoun/(c.convert(cur,'EUR',prx)))

    

In [49]:
r= rm.get_request("asset")

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [91]:
r[100]['LAST_CLOSE_VALUE']['value']

'359,34 EUR'

In [60]:
r[99]['CURRENCY']['value']

'USD'

In [96]:
data.T[100]

array(['359.34', 'EUR'],
      dtype='<U32')

In [103]:
def write_file(data, name):
    f1 = open(name, "w")
    if (len(data.shape) == 2):
        for i in range(data.shape[0]):
            for j in range(data.shape[1] - 1):
                f1.write(str(data[i][j]) + ' ')
            f1.write(str(data[i,data.shape[1] - 1]) + '\n')
    else:
        for j in range(data.shape[0] - 1):
                f1.write(str(data[i]) + ' ')
                f1.write(str(data[data.shape[0] - 1]) + '\n')
    f1.close()

In [98]:
cov = np.array([[a[i][j]  / np.sqrt(b[i] * b[j])for i in range(a.shape[0])]for j in range(a.shape[1]) ])
   

In [39]:
idss = rm.get_assetsids()
idss = idss.astype(np.int32)
rm.get_request("ratio")

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


[{'id': 15,
  'is_benchmark_needed': True,
  'is_percent': False,
  'name': 'Beta',
  'type': 'Ratio'},
 {'id': 19,
  'is_benchmark_needed': True,
  'is_percent': False,
  'name': 'Correlation',
  'type': 'Ratio'},
 {'id': 29,
  'is_benchmark_needed': False,
  'is_percent': True,
  'name': 'Exposition action',
  'type': 'Ratio'},
 {'id': 16,
  'is_benchmark_needed': False,
  'is_percent': True,
  'name': 'RDT',
  'type': 'Ratio'},
 {'id': 21,
  'is_benchmark_needed': False,
  'is_percent': True,
  'name': 'Rendement',
  'type': 'Ratio'},
 {'id': 17,
  'is_benchmark_needed': False,
  'is_percent': True,
  'name': 'Rendement_annualise',
  'type': 'Ratio'},
 {'id': 20,
  'is_benchmark_needed': False,
  'is_percent': False,
  'name': 'Sharpe',
  'type': 'Ratio'},
 {'id': 22,
  'is_benchmark_needed': False,
  'is_percent': True,
  'name': 'VaR historique',
  'type': 'Ratio'},
 {'id': 18,
  'is_benchmark_needed': False,
  'is_percent': True,
  'name': 'Volatilite',
  'type': 'Ratio'}]

In [44]:
asset_corr = []
self = rm
ids = idss[:10]
start = datetime.datetime(2012,1,1)
end = datetime.datetime(2017,6,1)


In [45]:
obj2 = {'ratio':[19],'asset':[int(ids[i]) for i in range(len(ids))],'benchmark':315,'start_date':start.isoformat(),'end_date':end.isoformat(),'frequency':None}
obj = json.dumps(obj2)
r = self.session.post(self.url + "/ratio/invoke", data = obj, verify = False)
dic = json.loads(r.text)
#a = [float((dic[str(ids[i])]['18']['value']).replace(",",'.')) for i in range(len(ids))]
       

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:794: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [47]:
dic[str(ids[0])]['19']['value'].replace(",",'.')

'0.999265785609'

## 1 ) Trivial Classificaion by best sharps

In [76]:
f1 = open("my_sharpes.txt","r")
sharpes = np.array(f1.readline().split()).astype(np.float32)
f1.close()
idx = np.argsort(sharpes)[::-1]
ids = idx[:20]
sh1 = rm.get_portfoliosh(ids,Ret_J=True, Sig_J=True)
sh2 = rm.get_portfoliosh(ids, Ret_J=True, Sig_J=False)
sh3 = rm.get_portfoliosh(ids, Ret_J=False, Sig_J=True)
sh4 = rm.get_portfoliosh(ids, Ret_J=False, Sig_J=False)
print("Use Jump Return + Jump Sigma  :", sh1)
print("Use Jump Return + 65 Sigma  :", sh2)
print("Use 65 Return +  Jump Sigma :", sh3)
print("Use 65 Return + 65 Sigma :", sh4)
print("Sum of sharpes :", sh1 + sh2 + sh3 + sh4)
rm.get_assets_quant(ids, rm.get_weights(ids), 10*1e6)

Use Jump Return + Jump Sigma  : 0.20161985138146446
Use Jump Return + 65 Sigma  : 80.65770457071
Use 65 Return +  Jump Sigma : 0.001624921225077248
Use 65 Return + 65 Sigma : 0.6535830034613018
Sum of sharpes : 81.51453234677784


AttributeError: 'RequestManadger' object has no attribute 'prx_cur'

# 2) Sharpe of sharpe classification

In [75]:
f2 = open("my_sharpes65.txt", "r")
data2 = f2.readlines() 
sharpes = np.array([np.array(data2[i].split()).astype(np.float64) for i in range(len(data2))])
f2.close()
eps = 1e-5
rfs = sharpes.min() - eps
resharpes = np.array([(sharpes[i].mean() - rfs)/np.var(sharpes[i]) for i in range(sharpes.shape[0])])
idx2 = np.argsort(resharpes)[::-1]
#print(idx2)
ids = idx2[:20]
sh1 = rm.get_portfoliosh(ids,Ret_J=True, Sig_J=True)
sh2 = rm.get_portfoliosh(ids, Ret_J=True, Sig_J=False)
sh3 = rm.get_portfoliosh(ids, Ret_J=False, Sig_J=True)
sh4 = rm.get_portfoliosh(ids, Ret_J=False, Sig_J=False)
print("Use Jump Return + Jump Sigma  :", sh1)
print("Use Jump Return + 65 Sigma  :", sh2)
print("Use 65 Return +  Jump Sigma :", sh3)
print("Use 65 Return + 65 Sigma :", sh4)
print("Sum of sharpes :", sh1 + sh2 + sh3 + sh4)

Use Jump Return + Jump Sigma  : 0.04610290946355646
Use Jump Return + 65 Sigma  : 24.107636373581098
Use 65 Return +  Jump Sigma : 0.001007848957492074
Use 65 Return + 65 Sigma : 0.32696442592667807
Sum of sharpes : 24.481711557928826


# 3) the most decorelated sharpe classification

In [47]:
f3 = open("my_assets.txt","r")
data1 = f3.readlines() 
assets_ret = np.array([np.array(data1[i].split()).astype(np.float64) for i in range(len(data1))])



In [49]:
#we reduce the assets by dividing by sigma as the correlation matrix would be exactly as covariance matrix because
#we want to use the property cov(x+y, z) = cov(x,z) + cov(y, z)
assets_ret_reduced = np.array([assets_ret[i]/np.sqrt(np.var(assets_ret[i])) for i in range(assets_ret.shape[0])])
idx3 = idx[:100]
b = assets_ret_reduced[idx3]
n = 20
btmp = assets_ret[0]
best_assets = [btmp]
# Sequence initialization
Xn0 = btmp
Xn1 = btmp
cor1 = np.corrcoef(b)
Zn0 = assets_ret_reduced[cor1[0].argmin()]
Zn1 = Zn0
b1 = np.array(b)
b2 = np.array(b)
index = 0
for i in range(n - 1):
    #print(b.shape)
    b1 = np.delete(b1, 0, axis = 0)
    b1 = b1.insert(b1,0, Xn1, axis = 0)
    
    b2 = np.delete(best_assets[-1])
    cor1 = np.corrcoef(b1)
    #print(cor)
    i = np.argmin(np.abs(cor[0]))
    assert(btmp.shape == b[i].shape),'error shape'
    btmp = btmp + b[i]
    #print(b.shape)
    #print(btmp.shape,"btmp")
    b = np.delete(b, 0, axis = 0)
    b = np.insert(b,0,btmp, axis = 0)

best_ever = b[indexs]


TypeError: delete() missing 1 required positional argument: 'obj'

In [42]:
f1 = open("ret_ass1.txt", "w")
for i in range(ret.shape[1] - 1):
    f1.write(str(ret[0][i]) + ' ')
f1.write(str(ret[0,ret.shape[1] - 1]) + '\n')
f1.close()
f1 = open("sh_ass1.txt", "w")
for i in range(sh.shape[1] - 1):
    f1.write(str(sh[0][i]) + ' ')
f1.write(str(sh[0,sh.shape[1] - 1]) + '\n')
f1.close()

15

In [107]:
ids = idx[:20]
w = np.array(W)
f1 = open("ret_ass1.txt", "r")
f2 = open("sh_ass1.txt", "r")
f3 = open("my_assets.txt","r")
rets = np.array(f1.readline().split()).astype(np.float32)[ids].reshape(20,1)
sh = np.array(f2.readline().split()).astype(np.float32)[ids].reshape(20,1)
data1 = f3.readlines() 
ret_cov = np.array([np.array(data1[i].split()).astype(np.float64) for i in range(len(data1))])[ids]
        
            

In [134]:
w= np.array(w)

In [139]:
Sigma = np.cov(ret_cov)
sigmap = np.sqrt(w.T.dot(Sigma.dot(w)))
rp =  rets.T.dot(w)
sharp = rp/sigmap
sharp

array([[ 32.30904133]])

In [49]:
idx2 = np.argsort(sh)[::-1]

In [50]:
idx2[:20]

array([444, 445,  67, 268, 319, 226,  19, 200, 101, 395, 289,  66,  72,
       514, 304,  41,  52, 334,  71, 147])

In [51]:
idx[:20]

array([501, 376, 145, 194,  53, 243,  47, 417, 396, 441,  95, 483, 245,
       251, 419, 523, 348,  80, 368, 330])

In [3]:
assets = []
sharpes = []
try:
    f1 = open("my_assets.txt", "r")
    f2 = open("my_sharpes.txt", "r")
    data1 = f1.readlines() 
    assets = np.array([np.array(data1[i].split()).astype(np.float64) for i in range(len(data1))])
    data2 = f2.readlines() 
    sharpes = np.array([np.array(data2[i].split()).astype(np.float64) for i in range(len(data2))])
    print('sucess')
except IOError:
    print('Error:file to open')
    a, s= rm.get_asset_vecrs(ids)
    assets = np.array([a[:,i] for i in range(a.shape[1])])
    sharpes = np.array([s[:,i] for i in range(s.shape[1])])
    f1 = open("my_assets.txt", "w")
    for i in range(assets.shape[0]):
        for j in range(assets.shape[1] - 1):
            f1.write(str(assets[i,j]) + " ")
        f1.write(str(assets[i,assets.shape[1] - 1]) + "\n")
    f2 = open("my_sharpes.txt", "w")
    for i in range(sharpes.shape[0]):
        for j in range(sharpes.shape[1] - 1):
             f2.write(str(sharpes[i,j]) + " ")
        f2.write(str(sharpes[i,sharpes.shape[1] - 1]) + "\n")

f2.close()
f1.close()

sucess


In [4]:
np.var(sharpes[0])

29.220085084891867

In [5]:
eps = 1e-5
rfs = sharpes.min() - eps
resharpes = np.array([(sharpes[i].mean() - rfs)/np.var(sharpes[i]) for i in range(sharpes.shape[0])])
idx = np.argsort(resharpes)

In [6]:

best_assets = assets[idx][::-1][:100]
#should reduce sigma before corcoeff


array([  1.18467823e+01,   1.22064971e+01,   1.95847333e+00,
         2.25029844e+00,   3.25950938e+00,   2.25554802e+00,
         3.11292798e+00,   4.65590236e+00,   4.57251751e+00,
         2.08150418e+00,   1.99891927e+00,   2.77507893e+00,
         2.04276435e+00,   1.76928627e+00,   4.07561745e+00,
         4.08213911e+00,   4.26448905e+00,   1.17220026e+00,
         1.22240112e+00,   1.29828034e+01,   4.46856253e+00,
         7.36003073e+00,   1.00316848e+01,   1.35819562e+01,
         8.11759030e-01,   2.66324535e+00,   4.49035113e+00,
         1.65852621e+00,   3.32229997e+00,   2.39657266e+00,
         2.31371154e+00,   1.74340093e+00,   2.74647653e+00,
         4.02843159e+00,   3.20075213e+00,   3.78573649e+00,
         4.01128291e+00,   1.31048090e+00,   5.75584761e+00,
         3.98966446e+00,   1.11928431e+01,   4.55214533e+00,
         4.38664855e+00,   8.23894169e+00,   3.95373158e+00,
         3.95165870e+00,   1.54698547e+00,   4.06182222e-01,
         2.47720246e+00,

# Trivial Version

In [129]:
sigma = np.cov(best_assets[:20])
RT = np.mean(best_assets[:20], axis=1).reshape(20,1)
#RT = rets.reshape(20,1).astype(np.double)
#sigma = np.cov(ret_cov)
#RT


array([[  1.40809965],
       [ 11.33072948],
       [  6.21949339],
       [  2.9272728 ],
       [  0.37968862],
       [  0.04191225],
       [ -0.56536359],
       [  7.40736055],
       [  0.47440132],
       [  1.61463416],
       [  4.17041206],
       [  2.56378436],
       [  5.24610615],
       [ -0.2702806 ],
       [  2.16455221],
       [  2.95806122],
       [  3.89030218],
       [  0.64064914],
       [  0.54479766],
       [  1.47922432]])

In [130]:
sigma = np.cov(best_assets[:20])
RT = np.mean(best_assets[:20], axis=1).reshape(20,1)
#RT = rets.reshape(20,1).astype(np.double)
#sigma = np.cov(ret_cov)
#RT
N = 20
P = matrix(sigma)
q = matrix(-2 * RT)
G = matrix(np.vstack((np.identity(N), -np.identity(N))),tc='d')
print("hh")
h = matrix(np.hstack((0.1 * np.ones(N), np.ones(N) * -0.01)), tc='d')
A = matrix(np.ones(N), (1,N)) 
b = matrix([1.0])
sol = solvers.qp(P, q, G, h, A, b)
W = sol['x']

hh
     pcost       dcost       gap    pres   dres
 0: -2.5598e+02 -4.0027e+01  2e+03  4e+01  2e-16
 1: -1.1112e+01 -1.8195e+01  6e+01  1e+00  4e-16
 2: -8.1877e+00 -1.4836e+01  7e+00  2e-16  4e-16
 3: -8.9751e+00 -9.8258e+00  9e-01  1e-16  3e-16
 4: -9.3710e+00 -9.4988e+00  1e-01  4e-16  1e-16
 5: -9.4297e+00 -9.4502e+00  2e-02  7e-17  2e-16
 6: -9.4380e+00 -9.4398e+00  2e-03  4e-16  2e-16
 7: -9.4390e+00 -9.4390e+00  2e-05  2e-16  2e-16
 8: -9.4390e+00 -9.4390e+00  2e-07  3e-16  2e-16
Optimal solution found.


In [132]:
print(np.array(W * 100).T)
print(idx[:20][::-1])
w = W

[[ 1.00000016  9.99999998  9.99999997  9.99999854  1.00000011  1.00000009
   1.00000006  9.99999997  1.00000012  1.00000024  9.9999998   9.00000165
   9.99999993  1.00000007  1.00000044  9.99999866  9.99999972  1.00000014
   1.00000013  1.00000023]]
[330 368  80 348 523 419 251 245 483  95 441 396 417  47 243  53 194 145
 376 501]


In [174]:
b = np.array([best_assets[i]/np.sqrt(np.var(best_assets[i])) for i in range(best_assets.shape[0])])
np.corrcoef(b)

array([[ 1.        ,  0.98797752,  0.09092401, ..., -0.00319322,
         0.02094697,  0.03845862],
       [ 0.98797752,  1.        ,  0.19480697, ...,  0.10558068,
         0.14190342,  0.15378604],
       [ 0.09092401,  0.19480697,  1.        , ...,  0.52487126,
         0.60065338,  0.63806002],
       ..., 
       [-0.00319322,  0.10558068,  0.52487126, ...,  1.        ,
         0.93370724,  0.95536563],
       [ 0.02094697,  0.14190342,  0.60065338, ...,  0.93370724,
         1.        ,  0.95275921],
       [ 0.03845862,  0.15378604,  0.63806002, ...,  0.95536563,
         0.95275921,  1.        ]])

In [175]:
b = best_assets[:50]
n = 20
btmp = best_assets[0]
indexs = [0]
for i in range(1, n):
    #print(b.shape)
    cor = np.corrcoef(b)
    #print(cor)
    i = np.argmin(np.abs(cor[0]))
    indexs.append(i)
    assert(btmp.shape == b[i].shape),'error shape'
    btmp = btmp + b[i]
    #print(b.shape)
    #print(btmp.shape,"btmp")
    b = np.delete(b, 0, axis = 0)
    b = np.insert(b,0,btmp, axis = 0)

best_ever = b[indexs]



In [176]:
best_ever.shape

(20, 65)

In [105]:

(cov == cov.T).astype(int).sum() / 528

528.0

In [109]:
f1 = open("my_ids.txt", "w")
for i in range(idss.shape[0] - 1):
    f1.write(str(idss[i]) + ' ')
f1.write(str(idss[i]) + '\n')
f1.close()

In [100]:
f1 = open("my_cov.txt", "w")
for i in range(cov.shape[0]):
    for j in range(cov.shape[1] - 1):
        f1.write(str(cov[i][j]) + ' ')
    f1.write(str(cov[0,cov.shape[1] - 1]) + '\n')
f1.close()

In [56]:
'jos' in dic

False

In [168]:
leng = len(one)
P = matrix(sigma)
q = matrix([-RT[i] for i in range(leng)])
G = matrix([[float(i ==j) - float((i - 4) == j) for j in range(leng)]  for i in range(2 * leng)])

h = matrix([0.1 if (i < leng) else -0.01 for i in range(leng)])
A = matrix(one, (1,leng)) 
b = matrix([1.0])
sol = solvers.qp(P, q, G, h, A, b)
W = sol['x']

     pcost       dcost       gap    pres   dres
 0: -1.3001e+00 -2.0788e+00  4e+01  7e+00  3e+00
 1: -6.4845e-01 -1.8228e+00  3e+00  3e-01  1e-01
 2: -5.6043e-01 -9.6503e-01  4e-01  6e-04  2e-04
 3: -8.6352e-01 -1.1279e+00  3e-01  1e-04  6e-05
 4: -9.4400e-01 -9.5258e-01  9e-03  4e-06  2e-06
 5: -9.5074e-01 -9.5083e-01  9e-05  4e-08  2e-08
 6: -9.5081e-01 -9.5081e-01  9e-07  4e-10  2e-10
Optimal solution found.
